# Component: Model I/O

In [1]:
!pip install langchain langchain-openai

  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB ? eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 57.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.2 kB ? eta -:--:--
   --------------------------------------- 894.2/894.2 kB 42.1 MB/s eta 0:00:00
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# .env에 키를 넣어놨는데 작동하지 않는 경우, 키가 정상적으로 있는지 확인하기
# import os
# os.getenv("OPENAI_API_KEY")

### PromptTemplate

In [4]:
# prompttemplate
from langchain import PromptTemplate

template = "{product}를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요."

prompt = PromptTemplate(
    template=template,
    input_variables=['product']
)

print(prompt.format(product="카메라"))

카메라를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요.


In [10]:
# FewShotPromptTemplate
from langchain import FewShotPromptTemplate

examples = [
    {'question': '2 + 2 는 무엇인가요?', 'answer': '2 + 2 = 4'},
    {'question': '3 + 4 는 무엇인가요?', 'answer': '3 + 4 = 7'}
]

example_prompt = PromptTemplate(
    template="Q: {question}\nA: {answer}",
    input_variables=['question', 'answer']
)

fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix='다음 계산 문제를 해결하세요.',
    suffix='Q: {question}은 무엇인가요?\nA:',
    input_variables=['question'],
)

print(fewshot_prompt.format(question="34 + 78"))

다음 계산 문제를 해결하세요.

Q: 2 + 2 는 무엇인가요?
A: 2 + 2 = 4

Q: 3 + 4 는 무엇인가요?
A: 3 + 4 = 7

Q: 34 + 78은 무엇인가요?
A:


In [ ]:
# ChatpromptTemplate - System, Human, AI  유형별 메세지 작성
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

sys_msg = SystemMessagePromptTemplate.from_template('당신은 똑똑한 챗봇입니다.')
hm_msg = HumanMessagePromptTemplate.from_template('질문: {question}')
msg = ChatPromptTemplate.from_messages([sys_msg, hm_msg])


msg.format_messages(question='AI를 배우려면 뭐부터 해야 하나요?')


[SystemMessage(content='당신은 똑똑한한 챗봇입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문: AI를 배우려면 뭐부터 해야 하나요?', additional_kwargs={}, response_metadata={})]

### OutputParser

In [ ]:
# outputparser 인스턴스 및 prompttemplate 인스턴스 생성성
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt_tpl = PromptTemplate(
    template='{subject} 5개의 팀을 알려주세요.\n형식지정: {format}',
    input_variables=['subject'],    # 사용자 입력 변수
    partial_variables={'format': format_instructions}   # 고정 설정 변수
)

In [17]:
# 프롬프트 생성
query = "한국의 프로축구팀"
prompt = prompt_tpl.format(subject=query)
print(prompt)

한국의 프로축구팀 5개의 팀을 알려주세요.
형식지정: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
# open ai api를 사용하기 위한 인스턴스 생성 및 요청 처리리
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0,
    max_tokens=2048
)

response = model.invoke(prompt)   # prompt의 질의를 보냄

In [21]:
print(response.content)
print(type(response.content))   # string

FC 서울, 전북 현대 모터스, 수원 삼성 블루윙즈, 울산 현대, 포항 스틸러스
<class 'str'>


In [22]:
print(output_parser.parse(response.content))
print(type(output_parser.parse(response.content)))  # string -> list

['FC 서울', '전북 현대 모터스', '수원 삼성 블루윙즈', '울산 현대', '포항 스틸러스']
<class 'list'>


### HuggingFace Model

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

endpoint = HuggingFaceEndpoint(
    repo_id="Bllossom/llama-3.2-Korean-Bllossom-3B",
    task="text-generation",
    max_new_tokens=1024
)

hf_model = ChatHuggingFace(
    llm=endpoint,
    verbose=True
)

hf_model.invoke('저는 어젯밤에 풋살을 했습니다. 저는 어젯밤에 무엇을 했을까요?')

In [26]:
from langchain.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([model, hf_model])
model_lab.compare('밤에 풋살하는 것의 효과를 알려주세요.')


Input:
밤에 풋살 하는 것의 효과를 알려주세요.

client=<openai.resources.chat.completions.completions.Completions object at 0x0000019F13C2CFA0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019F13C42160> root_client=<openai.OpenAI object at 0x0000019F13825310> root_async_client=<openai.AsyncOpenAI object at 0x0000019F13C2CF40> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') max_tokens=2048
밤에 풋살을 하는 것은 여러 가지 효과와 장점을 가지고 있습니다. 다음은 그 몇 가지입니다:

1. **체력 증진**: 풋살은 빠른 속도와 높은 강도의 운동으로, 심폐 지구력과 근력 향상에 도움을 줍니다. 밤에 운동을 하면 하루의 스트레스를 해소하고 체력을 키울 수 있습니다.

2. **사회적 상호작용**: 풋살은 팀 스포츠로, 친구나 동료와 함께하는 경우가 많습니다. 밤에 풋살을 하면 사회적 유대감을 강화하고, 팀워크를 향상시킬 수 있습니다.

3. **스트레스 해소**: 운동은 스트레스를 줄이는 데 효과적입니다. 밤에 풋살을 하면서 하루의 피로와 스트레스를 해소할 수 있습니다.

4. **기온 조절**: 여름철에는 낮보다 밤에 기온이 낮아 운동하기 더 쾌적할 수 있습니다. 더위로 인한 피로를 덜 느끼고 운동에 집중할 수 있습니다.

5. **정신적 집중력 향상**: 밤에 운동을 하면 하루의 일과를 마친 후 집중력을 높일 수 있습니다. 운동 중에는 정신적으로도 리프레시할 수 있습니다.

6. **수면 개선**: 규칙적인 운동은 수면

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://router.huggingface.co/hf-inference/models/Bllossom/llama-3.2-Korean-Bllossom-3B/v1/chat/completions (Request ID: Root=1-67cfac56-16a0b8717f8f46442d48c51c;a8f9b15c-5f01-4c63-b493-90f650ea68f7)

Model too busy, unable to get response in less than 60 second(s)